### ResNet

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
inp = tf.random_uniform([1 , 224 , 224 , 3])


In [3]:
def Layer(data , size , K , s , n ):
    
    with tf.variable_scope( n , reuse=tf.AUTO_REUSE ) as scope:
        
        out = tf.layers.conv2d(
        inputs  = data,
        filters = size,
        kernel_size = [K,K],
        strides=(s,s),
        padding = 'SAME',
        activation = None,
        name = n + 'Conv1')
        
        out = tf.layers.batch_normalization(inputs = out)
        
    return out 
    

In [4]:
def start(data , size2 , size1 , name , s = 2 ):
        
    out = Layer(data , size1 , 1 , s , name + 'CONV1')
    out = tf.nn.relu(out)
        
    out = Layer(out , size1 , 3 , 1 , name + 'Conv2')
    out = tf.nn.relu(out)
        
    out = Layer(out , size2 , 1 , 1 , name + 'Conv3' )
     

    data = Layer(data , size2 , 1 , s , name + 'DATA')
    
    out =np.add(out , data) 
    out = tf.nn.relu(out)
    
    return out


In [5]:
def res_block(data , size1 , size2 , name ):
    
     
    out = Layer(data , size1 , 1 , 1 , name + 'Conv1')
    out = tf.nn.relu(out)
    out = Layer(out , size1 , 3 , 1 , name + 'Conv2')
    out = tf.nn.relu(out)
    out = Layer(out , size2 , 1 , 1 , name + 'Conv3' )
    
    out =np.add(out , data) 
    
    out = tf.nn.relu(out)
    
    return out
    
    

In [6]:
def loop( n , data , s1 , s2 , name ):
    
    for i in range (n):
        data = res_block(data , s1 , s2 , name.format(i))
        
    return data
        

In [7]:
def model(data):
    
    out=tf.layers.conv2d(
    inputs=data,
    filters=64,
    kernel_size=[7,7],
    strides=(2,2),
    activation=None,
    padding='SAME',
    reuse=tf.AUTO_REUSE,
    name='CONV1')
    
    out = tf.layers.batch_normalization(inputs = out)
    
    print(out)  #expected [112 , 112]
    
    out=tf.layers.max_pooling2d(
    inputs=out,
    pool_size=[3,3],
    strides=(2,2),
    padding='SAME',
    name='MAXPOOL')
    
    print(out)  #expected [56 , 56]

    #Res block1
    
    out = start(out , 256 , 64 , 'BLOCK_1_OUT' , 1 )
        
    with tf.variable_scope('Block1') as scope:
        
        out = loop(2 , out , 64 , 256 , 'Block1_{}')
            
    print(out)  #expected [56 , 56]
    
    #Res block2
    
    out = start(out , 512 , 128 , 'BLOCK_2_OUT' )
        
    with tf.variable_scope('Block2') as scope:
        
        out = loop(7 , out , 128 , 512 , 'Block2_{}')

            
    print(out)  #expected [28 , 28]
    
    #Res block3
    
    out = start(out , 1024 , 256 , 'BLOCK_3_OUT' )
        
    with tf.variable_scope('Block3') as scope:
        
        out = loop(35 , out , 256 , 1024 , 'Block3_{}')
    
    print(out)  #expected [14 , 14]
    
    #Res block4
    
    out = start(out , 2048 , 512 , 'BLOCK_4_OUT' )
        
    with tf.variable_scope('Block4') as scope:
        
        out = loop(2 , out , 512 , 2048 , 'Block4_{}')
            
    print(out)  #expected [7 , 7]
    
    out = tf.keras.layers.GlobalAveragePooling2D()(out)
    
    out = tf.layers.dense(
    inputs=out,
    units=1000,
    activation=None,
    reuse=tf.AUTO_REUSE,
    name='OUT')
    
    return out

In [8]:
ot = model(inp)
ot

Tensor("batch_normalization/FusedBatchNorm:0", shape=(1, 112, 112, 64), dtype=float32)
Tensor("MAXPOOL/MaxPool:0", shape=(1, 56, 56, 64), dtype=float32)
Tensor("Block1/Relu_5:0", shape=(1, 56, 56, 256), dtype=float32)
Tensor("Block2/Relu_20:0", shape=(1, 28, 28, 512), dtype=float32)
Tensor("Block3/Relu_104:0", shape=(1, 14, 14, 1024), dtype=float32)
Tensor("Block4/Relu_5:0", shape=(1, 7, 7, 2048), dtype=float32)


<tf.Tensor 'OUT/BiasAdd:0' shape=(1, 1000) dtype=float32>